In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import sys
sys.path.append('../')
from dataset_formatter import UniMediaDatasetFormatter

import re
from pprint import pprint
from sklearn.utils import shuffle # For shuffling datasets

In [2]:
import cv2 # Run `pip install opencv-python` to install
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib qt5

In [3]:
def parse_filename(filename):
  """
  Args:
    filename: e.g. 'person01_running_d1_uncomp.avi'
  Returns:
    e.g. 'person01_running_d1', '01', 'running', 'd1'
  """
  FILENAME_PATTERN = r'(person(\d+)_(\w+)_(d\d))_uncomp.avi'
  res = re.match(FILENAME_PATTERN, filename)
  if res:
    basename, person_id, action, remark = res.groups()
    person_id = int(person_id)
    return basename, person_id, action, remark
  else:
    raise ValueError("Filename not in good pattern!")

def get_kth_info_df(kth_dir, tmp_dir='/tmp/', from_scratch=False):
  csv_filepath = os.path.join(tmp_dir, 'kth_files_info.csv')
  if not from_scratch and os.path.isfile(csv_filepath):
    kth_df = pd.read_csv(csv_filepath)
    print("Successfully loaded existing info table. Now life is easier.")
    return kth_df
  else:
    print("Couldn't load existing info table. Now building from scatch...")
    train_id = [11, 12, 13, 14, 15, 16, 17, 18]
    valid_id = [19, 20, 21, 23, 24, 25, 1, 4]
    test_id = [22, 2, 3, 5, 6, 7, 8, 9, 10]
    train_id = train_id + valid_id # Merge training set and validation set

    def get_subset(person_id):
      """
      Returns:
        'train' or 'test'
      """
      if person_id in train_id:
        return 'train'
      elif person_id in test_id:
        return 'test'
      else:
        raise ValueError(f"Wrong person id {person_id}!")

    path = kth_dir
    li = []
    for dirpath, dirnames, filenames in os.walk(path):
      for filename in filenames:
        if filename.endswith('.avi'):
          basename, person_id, action, remark = parse_filename(filename)
          video_filepath = os.path.join(dirpath, filename)
          subset = get_subset(person_id)
          li.append((basename, person_id, action, remark, video_filepath, subset))
    kth_df = pd.DataFrame({'basename':        [x[0] for x in li],
                           'person_id':       [x[1] for x in li],
                           'action':          [x[2] for x in li],
                           'remark':          [x[3] for x in li],
                           'video_filepath':  [x[4] for x in li],
                           'subset':          [x[5] for x in li]})
    kth_df['action'] = kth_df['action'].astype('category')
    kth_df['action_num'] = kth_df['action'].cat.codes
    kth_df['remark'] = kth_df['remark'].astype('category')
    kth_df['remark_num'] = kth_df['remark'].cat.codes

    kth_df.to_csv(csv_filepath, index=False)
    return kth_df

In [4]:
def video_to_3d_features(video_filepath, resize=(1,1)):
  cap = cv2.VideoCapture(video_filepath)
  features = []
  printed = False
  while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
      break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape
    fx, fy = resize
    new_height = int(fx * height)
    new_width = int(fx * width)
    if not printed and resize != (1,1):
      print('Resizing video from original size {} to new size {}...'.format((height, width), (new_height, new_width)))
      printed = True
    gray_resized = cv2.resize(gray, (new_width, new_height))
    gray_flattened = gray_resized.flatten()
    features.append(gray_flattened)
  cap.release()
  cv2.destroyAllWindows()
  features = np.array(features)
  return features

def play_video_from_file(filename):
  fig, ax = plt.subplots()
  vidcap = cv2.VideoCapture(filename)
  success,image = vidcap.read()
  image = np.array(image).astype(float)
  screen = plt.imshow(image)
  def init():  # only required for blitting to give a clean slate.
      screen.set_data(np.empty(image.shape))
      return screen,
  def animate(i):
      if vidcap.isOpened():
        success,image = vidcap.read()
        image = np.array(image).astype(float)
        screen.set_data(image)
      return screen,
  ani = animation.FuncAnimation(
      fig, animate, init_func=init, interval=40, blit=True, save_count=50, repeat=False) # interval=40 because 25fps
  plt.show()
  return ani, plt

def play_video_from_features(features, row_count=120, col_count=160, interval=80, resize=(1,1)):
  fig, ax = plt.subplots()
  new_row_count = int(row_count * resize[0])
  new_col_count = int(col_count * resize[1])
  print('Playing with {} rows and {} columns.'.format(new_row_count, new_col_count))
  image = features[0].reshape((new_row_count, new_col_count))
  screen = plt.imshow(image, cmap='gray')
  def init():  # only required for blitting to give a clean slate.
      screen.set_data(np.empty(image.shape))
      return screen,
  def animate(i):
      if i < len(features):
        image = features[i].reshape((new_row_count, new_col_count))
        screen.set_data(image)
      return screen,
  ani = animation.FuncAnimation(
      fig, animate, init_func=init, interval=interval, blit=True, save_count=50, repeat=False) # interval=40 because 25fps
  plt.show()
  return ani, plt

In [5]:
kth_dir = '../../raw_datasets/video/kth/'
df = get_kth_info_df(kth_dir)

Successfully loaded existing info table. Now life is easier.


In [6]:
filepath = np.random.choice(df['video_filepath'])
print('filepath:', filepath)

filepath: ../../raw_datasets/video/kth/handwaving/person10_handwaving_d4_uncomp.avi


In [7]:
resize = (0.5,0.5)
features = video_to_3d_features(filepath, resize=resize)
print(features.shape)
play_video_from_features(features, interval=40, resize=resize)

Resizing video from original size (120, 160) to new size (60, 80)...
(532, 4800)
Playing with 60 rows and 80 columns.


(<matplotlib.animation.FuncAnimation at 0x1304202e8>,
 <module 'matplotlib.pyplot' from '/Users/evariste/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py'>)

In [8]:
df

,action,basename,person_id,remark,subset,video_filepath,action_num,remark_num
0,running,person11_running_d1,11,d1,train,../../raw_datasets/video/kth/running/person11_...,4,0
1,running,person16_running_d2,16,d2,train,../../raw_datasets/video/kth/running/person16_...,4,1
2,running,person21_running_d4,21,d4,train,../../raw_datasets/video/kth/running/person21_...,4,3
3,running,person07_running_d1,7,d1,test,../../raw_datasets/video/kth/running/person07_...,4,0
4,running,person08_running_d2,8,d2,test,../../raw_datasets/video/kth/running/person08_...,4,1
5,running,person19_running_d1,19,d1,train,../../raw_datasets/video/kth/running/person19_...,4,0
6,running,person15_running_d3,15,d3,train,../../raw_datasets/video/kth/running/person15_...,4,2
7,running,person05_running_d4,5,d4,test,../../raw_datasets/video/kth/running/person05_...,4,3
8,running,person13_running_d4,13,d4,train,../../raw_datasets/video/kth/running/person13_...,4,3
9,running,person03_running_d3,3,d3,test,../../raw_datasets/video/kth/running/person03_...,4,2
